In [1]:
import dspy
from datasets import load_dataset
import os

In [2]:
# Configure Ollama Language Model for DSPy
# Prerequisites: 
# 1. Install Ollama: curl -fsSL https://ollama.ai/install.sh | sh
# 2. Run model: ollama run llama3.2:1b (or your preferred model)

import dspy

# Configure Ollama LM using DSPy's official format
ollama_llm = dspy.LM(
    model='ollama_chat/gemma3:4b',  # Format: ollama_chat/{model_name}
    api_base='http://localhost:11434',  # Ollama default endpoint
    api_key='',  # Empty string for local Ollama
    max_tokens=65536,
    temperature=1.0
)

# Set as default LM
dspy.configure(lm=ollama_llm)

print("✅ Ollama LM configured successfully!")
print(f"Model: {ollama_llm.model}")
print("🔄 Make sure Ollama is running: ollama run qwen3:8b")

✅ Ollama LM configured successfully!
Model: ollama_chat/gemma3:4b
🔄 Make sure Ollama is running: ollama run qwen3:8b


In [5]:
open_router_lm = dspy.LM('openrouter/openai/gpt-4.1-nano', 
                          api_key=os.getenv('openrouter_api_key'), 
                          api_base='https://openrouter.ai/api/v1',
                          max_tokens=65536,
                          temperature=1.0)

dspy.configure(lm=open_router_lm)

reflection_lm = dspy.LM('openrouter/meta-llama/llama-4-scout', 
                          api_key=os.getenv('openrouter_api_key'), 
                          api_base='https://openrouter.ai/api/v1',
                          max_tokens=65536,
                          temperature=1.0)

In [6]:
train_split = load_dataset("AI-MO/NuminaMath-1.5")['train']

In [7]:
def is_numeric_answer(answer):
    try:
        int(answer)  # Try converting string to int number
        return True
    except (ValueError, TypeError):
        return False

In [8]:
# keep only the samples where its ['answer'] key is int or float, do it modular and fast.
train_split = train_split.filter(lambda x: is_numeric_answer(x['answer']))

Filter:   0%|          | 0/896215 [00:00<?, ? examples/s]

In [9]:
print(train_split[12]['answer'])

1898


In [10]:
def init_dataset(train_split_ratio=None, test_split_ratio=None, val_split_ratio=None, sample_fraction=1.0):
    if train_split_ratio is None:
        train_split_ratio = 0.5
    if test_split_ratio is None:
        test_split_ratio = 0.45
    if val_split_ratio is None:
        val_split_ratio = 0.05
    assert (train_split_ratio + test_split_ratio + val_split_ratio) == 1.0, "Ratios must sum to 1.0"

    train_split = load_dataset("AI-MO/NuminaMath-1.5")['train']
    # keep only the samples where its ['answer'] key is int or float.
    train_split = train_split.filter(lambda x: is_numeric_answer(x['answer']))
    train_split = [
        dspy.Example({
            "problem": x['problem'],
            'solution': x['solution'],
            'answer': x['answer'],
        }).with_inputs("problem")
        for x in train_split
    ]
    import random
    random.Random(0).shuffle(train_split)
    tot_num = len(train_split)
    print(f"Total number of examples after filtering: {tot_num}")

    if sample_fraction < 1.0:
        sample_num = int(tot_num * sample_fraction)
        train_split = train_split[:sample_num]
        tot_num = sample_num
        print(f"Sampled down to {sample_num} examples.")
    
    train_set = train_split[:int(train_split_ratio * tot_num)]
    val_set = train_split[int(train_split_ratio * tot_num):int((train_split_ratio + val_split_ratio) * tot_num)]
    test_set = train_split[int((train_split_ratio + val_split_ratio) * tot_num):]

    return train_set, val_set, test_set

In [11]:
train_set, val_set, test_set = init_dataset(sample_fraction=0.01)

len(train_set), len(val_set), len(test_set)

Filter:   0%|          | 0/896215 [00:00<?, ? examples/s]

Total number of examples after filtering: 298564
Sampled down to 2985 examples.


(1492, 149, 1344)

In [12]:
print("Problem:")
print(train_set[0]['problem'])
print("\n\nSolution:")
print(train_set[0]['solution'])
print("\n\nAnswer:")
print(train_set[0]['answer'])

Problem:
The graph of the equation $y = \frac{x}{x^3 + Ax^2 + Bx + C}$, where $A,B,C$ are integers, has vertical asymptotes at $x = -1, 3, 4$. Find $A + B + C$.


Solution:
Given the vertical asymptotes at $x = -1, 3, 4$, the denominator of the function can be formulated as: 
\[ x^3 + Ax^2 + Bx + C = (x + 1)(x - 3)(x - 4) \]

First, expand the product:
\[ (x + 1)(x - 3)(x - 4) \]
\[ = (x + 1)(x^2 - 7x + 12) \]
\[ = x^3 - 7x^2 + 12x + x^2 - 7x + 12 \]
\[ = x^3 - 6x^2 + 5x + 12 \]

Thus, we have $A = -6$, $B = 5$, and $C = 12$, so:
\[ A + B + C = -6 + 5 + 12 = \boxed{11} \]


Answer:
11


In [13]:
print(test_set[0]['problem'])
print("\n\nAnswer:")
print(test_set[0]['answer'])

A [i]Beaver-number[/i] is a positive 5 digit integer whose digit sum is divisible by 17. Call a pair of [i]Beaver-numbers[/i] differing by exactly $1$ a [i]Beaver-pair[/i]. The smaller number in a [i]Beaver-pair[/i] is called an [i]MIT Beaver[/i], while the larger number is called a [i]CIT Beaver[/i]. Find the positive difference between the largest and smallest [i]CIT Beavers[/i] (over all [i]Beaver-pairs[/i]).


Answer:
79200


In [19]:
class GenerateResponse(dspy.Signature):
    """Solve the problem and provide the answer in the correct format."""
    problem = dspy.InputField()
    answer = dspy.OutputField()

program = dspy.ChainOfThought(GenerateResponse)

In [20]:
def metric(example, prediction, trace=None, pred_name=None, pred_trace=None):
    correct_answer = int(example['answer'])
    try:
        llm_answer = int(prediction.answer)
    except ValueError as e:
        return 0
    return int(correct_answer == llm_answer)

In [21]:
import dspy
evaluate = dspy.Evaluate(
    devset=test_set,
    metric=metric,
    num_threads=32,
    display_table=True,
    display_progress=True
)

evaluate(program)

Average Metric: 208.00 / 471 (44.2%):  35%|███▍      | 470/1344 [02:23<01:29,  9.73it/s]

2025/09/22 14:43:26 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 569.00 / 1344 (42.3%): 100%|██████████| 1344/1344 [08:19<00:00,  2.69it/s]

2025/09/22 14:49:21 INFO dspy.evaluate.evaluate: Average Metric: 569 / 1344 (42.3%)


,problem,solution,example_answer,reasoning,pred_answer,metric
0,A [i]Beaver-number[/i] is a positive 5 digit integer whose digit s...,1. **Understanding the Problem:** - A *Beaver-number* is a 5-digit...,79200,A Beaver-number is defined as a five-digit integer whose digit sum...,81080,
1,Two passenger trains start at the same hour in the day from two di...,Let's denote the distance travelled by the slower train (16 kmph) ...,60,Let the two trains start at the same time from two stations that a...,60 km,
2,Mcdonald is planning to open up a farm that provides eggs to the c...,Let's denote the number of eggs Ben needs per week as B. Since Ked...,14,Let's define the variables: - Saly needs 10 eggs per week. - Ben n...,Ben needs 14 eggs per week.,
3,Given that the arithmetic sequence ${a_n}$ has a sum of its first ...,"Since the sequence ${a_n}$ is an arithmetic sequence, it follows t...",15,We are given an arithmetic sequence \(\{a_n\}\) with sum of the fi...,15,✔️ [1]
4,The ratio of spinsters to cats is 2 to 9. If there are 42 more cat...,Let the number of spinsters be represented by S and the number of ...,12,Let's denote the number of spinsters as S and the number of cats a...,12,✔️ [1]
...,...,...,...,...,...,...
1339,"If $\begin{vmatrix} a & b \\ c & d \end{vmatrix} = 6,$ then find \...","Given that $\begin{vmatrix} a & b \\ c & d \end{vmatrix} = 6,$ it ...",12,Given the original determinant \(\begin{vmatrix} a & b \\ c & d \e...,12,✔️ [1]
1340,Valentina bought a foot long burger and shared half with his broth...,"If Valentina bought a foot long burger, that means the burger is 1...",6,"Valentina bought a foot-long burger, which is 12 inches long. She ...",Each person’s share is 6 inches.,
1341,"In a sequence, 1 = 6, 2 = 12, 3 = 18, 4 = 24, and 5 = some value. ...",The sequence given is: 1 = 6 2 = 12 3 = 18 4 = 24 5 = ? 6 = 1 From...,30,"The sequence given is: 1 = 6, 2 = 12, 3 = 18, 4 = 24. Observing th...",30,✔️ [1]
1342,The value of $x$ that satisfies $\binom{x+1}{x-4} = \frac{7}{15}P^...,**Analysis** This question examines the formulas for combinations ...,10,"Given the equation \(\binom{x+1}{x-4} = \frac{7}{15} P_{x+1}^3\), ...",10,✔️ [1]


EvaluationResult(score=42.34, results=<list of 1344 results>)

In [25]:
# SYSTEMATIC DEBUGGING - Step 1: Test program on single example (FIXED)
print("=== STEP 1: Testing program on single example ===")
test_example = test_set[0]
print(f"Input problem: {test_example.problem[:100]}...")
print(f"Expected answer: {test_example.answer}")

try:
    # FIX: Use keyword argument matching signature field name
    prediction = program(problem=test_example.problem)
    print(f"Program prediction: {prediction}")
    print(f"Prediction answer: {prediction.answer}")
    print(f"Prediction type: {type(prediction.answer)}")
    print("✅ Program works!")
except Exception as e:
    print(f"❌ Program failed: {e}")
    import traceback
    traceback.print_exc()

=== STEP 1: Testing program on single example ===
Input problem: A [i]Beaver-number[/i] is a positive 5 digit integer whose digit sum is divisible by 17. Call a pair...
Expected answer: 79200
Program prediction: Prediction(
    reasoning='A Beaver-number is defined as a five-digit integer whose digit sum is divisible by 17. A Beaver-pair consists of two Beaver-numbers differing by exactly 1, with the smaller called an MIT Beaver and the larger a CIT Beaver. We need to find the range of CIT Beavers across all such pairs, specifically the difference between the largest and smallest CIT Beavers.\n\nTo analyze this, consider two consecutive integers n and n+1 differing by 1. For both to be Beaver-numbers, their digit sums must both be divisible by 17 (since the sum of digits of n and n+1 must both satisfy this property).\n\nLet s(n) be the digit sum of n, and s(n+1) be that of n+1. The difference:\n\ns(n+1) - s(n) = (sum of digits of n+1) - (sum of digits of n)\n\nAdding 1 to n affects the

In [26]:
def metric_with_feedback(example, prediction, trace=None, pred_name=None, pred_trace=None):
    correct_answer = int(example['answer'])
    written_solution = example.get('solution', '')
    try:
        llm_answer = int(prediction.answer)
    except ValueError as e:
        feedback_text = f"The final answer must be a valid integer and nothing else. You responded with '{prediction.answer}', which couldn't be parsed as a python integer. Please ensure your answer is a valid integer without any additional text or formatting."
        feedback_text += f" The correct answer is '{correct_answer}'."
        if written_solution:
            feedback_text += f" Here's the full step-by-step solution:\n{written_solution}\n\nThink about what takeaways you can learn from this solution to improve your future answers and approach to similar problems and ensure your final answer is a valid integer."
        return dspy.Prediction(score=0, feedback=feedback_text)

    score = int(correct_answer == llm_answer)

    feedback_text = ""
    if score == 1:
        feedback_text = f"Your answer is correct. The correct answer is '{correct_answer}'."
    else:
        feedback_text = f"Your answer is incorrect. The correct answer is '{correct_answer}'."
    
    if written_solution:
        feedback_text += f" Here's the full step-by-step solution:\n{written_solution}\n\nThink about what takeaways you can learn from this solution to improve your future answers and approach to similar problems."

    return dspy.Prediction(score=score, feedback=feedback_text)

In [29]:
from dspy import GEPA

optimizer = GEPA(
    metric=metric_with_feedback,
    auto="heavy",
    num_threads=32,
    track_stats=True,
    reflection_minibatch_size=16,
    track_best_outputs=True,
    add_format_failure_as_feedback=True,
    reflection_lm=reflection_lm
)


In [30]:
optimized_program = optimizer.compile(
    program,
    trainset=train_set,
    valset=val_set,
)

2025/09/22 15:29:11 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 2078 metric calls of the program. This amounts to 1.27 full evals on the train+val set.
2025/09/22 15:29:11 INFO dspy.teleprompt.gepa.gepa: Using 149 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.
2025/09/22 15:29:11 INFO dspy.evaluate.evaluate: Average Metric: 65.0 / 149 (43.6%)
2025/09/22 15:29:11 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.436241610738255
2025/09/22 15:29:11 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.436241610738255
2025/09/22 15:29:11 WARNING dspy.primitives.module: Calling module.forward(...) on ChainOfThought directly is discouraged. Please use module(...) instead.


Average Metric: 5.00 / 16 (31.2%): 100%|██████████| 16/16 [00:00<00:00, 924.30it/s]

2025/09/22 15:29:11 INFO dspy.evaluate.evaluate: Average Metric: 5.0 / 16 (31.2%)
2025/09/22 15:29:11 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for predict: ### Instruction

Solve the given problem and provide the answer in the correct format. 

To ensure accuracy, consider the following guidelines:

1. **Read and Understand the Problem**: Carefully read the problem statement and identify the key components, including any specific constraints or requirements.

2. **Provide Detailed Reasoning**: Offer a step-by-step explanation of your thought process and calculations. This will help in ensuring that the approach is correct and easy to follow.

3. **Use Correct Mathematical Notation**: Ensure that all mathematical expressions and equations are clearly written and correctly formatted.

4. **Check for Common Mistakes**: Verify the calculations and reasoning to avoid common mistakes, such as incorrect unit conversions or miscalculations.

5. **Rationalize Denominators 

2025/09/22 15:29:12 INFO dspy.evaluate.evaluate: Average Metric: 70.0 / 149 (47.0%)
2025/09/22 15:29:12 INFO dspy.teleprompt.gepa.gepa: Iteration 1: New program is on the linear pareto front
2025/09/22 15:29:12 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Full valset score for new program: 0.4697986577181208
2025/09/22 15:29:12 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Full train_val score for new program: 0.4697986577181208
2025/09/22 15:29:12 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Individual valset scores for new program: [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0]
2025/09/22 15

Average Metric: 9.00 / 16 (56.2%): 100%|██████████| 16/16 [00:00<00:00, 689.43it/s]

...

2025/09/22 15:49:17 INFO dspy.evaluate.evaluate: Average Metric: 8.0 / 16 (50.0%)


2025/09/22 15:49:23 INFO dspy.teleprompt.gepa.gepa: Iteration 18: Proposed new text for predict: ### Instruction
Solve the given problem and provide the answer in the correct format. Ensure the final answer is a valid integer without any additional text or formatting. If there are multiple parts to the problem, provide a clear and concise solution for each part. Consider the context and domain-specific information that might be necessary to solve the task accurately.

The tasks may involve calculations, logical reasoning, or problem-solving strategies. Use the information provided in the examples and feedback to enhance your understanding of the tasks and provide accurate solutions.

In case of sequence or series problems, ensure you provide a clear step-by-step solution. For problems involving geometry or algebra, include relevant formulas or theorems used in the solution.

In your final answer, do not include units or additional descriptions unless specifically required by the task.


Average Metric: 10.00 / 16 (62.5%): 100%|██████████| 16/16 [00:18<00:00,  1.13s/it]

2025/09/22 15:51:40 INFO dspy.evaluate.evaluate: Average Metric: 10.0 / 16 (62.5%)


2025/09/22 15:51:45 INFO dspy.teleprompt.gepa.gepa: Iteration 19: Proposed new text for predict: ### Instruction
Solve the given problem and provide the answer in the correct format. Ensure the final answer is a valid response without any additional text or formatting. If there are multiple parts to the problem, provide a clear and concise solution for each part. Consider the context and domain-specific information that might be necessary to solve the task accurately.

The tasks may involve calculations, logical reasoning, or problem-solving strategies. Use the information provided in the examples and feedback to enhance your understanding of the tasks and provide accurate solutions.

In case of sequence or series problems, ensure you provide a clear step-by-step solution. For problems involving geometry or algebra, include relevant formulas or theorems used in the solution.

In your final answer, do not include units or additional descriptions unless specifically required by the task.

In [31]:
print(optimized_program.predict.signature.instructions)

### Instruction
Solve the given problem and provide the answer in the correct format. Ensure the final answer is a valid integer without any additional text or formatting. If there are multiple parts to the problem, provide a clear and concise solution for each part. Consider the context and domain-specific information that might be necessary to solve the task accurately.

The tasks may involve calculations, logical reasoning, or problem-solving strategies. Use the information provided in the examples and feedback to enhance your understanding of the tasks and provide accurate solutions.

In case of sequence or series problems, ensure you provide a clear step-by-step solution. For problems involving geometry or algebra, include relevant formulas or theorems used in the solution.

In your final answer, do not include units or additional descriptions unless specifically required by the task.

### Examples and Feedback for Reference:
- Provided for context and to improve future responses.

In [32]:
evaluate(optimized_program)

Average Metric: 860.00 / 1344 (64.0%): 100%|██████████| 1344/1344 [07:05<00:00,  3.16it/s]

2025/09/22 15:59:11 INFO dspy.evaluate.evaluate: Average Metric: 860 / 1344 (64.0%)


,problem,solution,example_answer,reasoning,pred_answer,metric
0,A [i]Beaver-number[/i] is a positive 5 digit integer whose digit s...,1. **Understanding the Problem:** - A *Beaver-number* is a 5-digit...,79200,A Beaver-number is a five-digit integer whose digits sum to a mult...,80253,
1,Two passenger trains start at the same hour in the day from two di...,Let's denote the distance travelled by the slower train (16 kmph) ...,60,Let the two trains start simultaneously from two stations separate...,60,✔️ [1]
2,Mcdonald is planning to open up a farm that provides eggs to the c...,Let's denote the number of eggs Ben needs per week as B. Since Ked...,14,The total eggs needed in a month are 124. Saly needs 10 eggs per w...,14,✔️ [1]
3,Given that the arithmetic sequence ${a_n}$ has a sum of its first ...,"Since the sequence ${a_n}$ is an arithmetic sequence, it follows t...",15,Given the arithmetic sequence \( a_n \) with sum of first \( n \) ...,15,✔️ [1]
4,The ratio of spinsters to cats is 2 to 9. If there are 42 more cat...,Let the number of spinsters be represented by S and the number of ...,12,"Let the number of spinsters be 2k and the number of cats be 9k, as...",12,✔️ [1]
...,...,...,...,...,...,...
1339,"If $\begin{vmatrix} a & b \\ c & d \end{vmatrix} = 6,$ then find \...","Given that $\begin{vmatrix} a & b \\ c & d \end{vmatrix} = 6,$ it ...",12,Given the determinant \(\begin{vmatrix} a & b \\ c & d \end{vmatri...,12,✔️ [1]
1340,Valentina bought a foot long burger and shared half with his broth...,"If Valentina bought a foot long burger, that means the burger is 1...",6,"Valentina bought a foot-long burger, which is 12 inches. She share...",6,✔️ [1]
1341,"In a sequence, 1 = 6, 2 = 12, 3 = 18, 4 = 24, and 5 = some value. ...",The sequence given is: 1 = 6 2 = 12 3 = 18 4 = 24 5 = ? 6 = 1 From...,30,"The sequence provided is: 1 = 6, 2 = 12, 3 = 18, 4 = 24. We observ...",30,✔️ [1]
1342,The value of $x$ that satisfies $\binom{x+1}{x-4} = \frac{7}{15}P^...,**Analysis** This question examines the formulas for combinations ...,10,We are given the equation \(\binom{x+1}{x-4} = \frac{7}{15} P_{x+1...,3,


EvaluationResult(score=63.99, results=<list of 1344 results>)